In [10]:
# Import required libraries
import pandas as pd
import glob
import numpy as np
from datetime import datetime as dt_obj

# Function to convert into date_time object for max time
def to_date_max(tstr1, tstr2):
    year   = int(tstr1[:4])
    month  = int(tstr1[5:7])
    day    = int(tstr1[8:10])
    hour   = int(tstr2[:2])
    minute = int(tstr2[2:4])
    return dt_obj(year, month, day, hour, minute)

In [11]:
%%time
# Create events dataset
DF = pd.DataFrame(columns=['Event', 'Begin', 'Max', 'End', 'Obs', 'Q', 'Type', 'Loc/Frq', 'Particulars', 'Reg'])
txt_files = glob.glob('Events/*.txt')

# Opening daily event files
for files in txt_files:
    with open(files, 'r') as f:
        f.readline()
        f.readline()
        date = f.readline()[7:17]
        f.close()
    #print(files)
    data = np.genfromtxt(files, dtype=str, skip_header=3, delimiter=(8, 8, 8, 8, 6, 3, 6, 9, 19, 5), autostrip=True)
    df = pd.DataFrame(data, columns=['Event', 'Begin', 'Max', 'End', 'Obs', 'Q', 'Type', 'Loc/Frq', 'Particulars', 'Reg'])
    df['Date'] = [date for i in range(len(df.Reg))]
    DF = pd.concat([DF, df], ignore_index=True)

# Filtering duplicates
DF = DF.drop_duplicates()
DF = DF.loc[DF['Event'] != '']

# Filtering for invalid values
temp_list = []
date = [i for i in DF.Date]
time = [i for i in DF.Begin]
Max = [i for i in DF.Max]
End = [i for i in DF.End]
# We filter invalid value in timestamp to create Date column which will be used as timestamp of the event
# First we look at Max time. If the value is '////' (invalid) or its str lenght != 4, replace it with End time
# If End time also invalid, replace it with '0000'
for i in range(len(time)):
    if len(time[i]) != 4 or time[i] == '////' or int(time[i]) > 60:
        if len(Max[i]) == 4 and Max[i] != '////':
            time[i] = Max[i]
        elif len(End[i]) == 4 and End[i] != '////':
            time[i] = End[i]
        else:
            time[i] = '0000'
        
for i in range(len(date)):
    date_time = to_date_max(date[i], time[i])
    temp_list.append(date_time)
DF['Date'] = temp_list

# Saving the event list as csv
DF.to_csv('All NOAA Notes.csv')
DF

Wall time: 59.3 s


,Event,Begin,Max,End,Obs,Q,Type,Loc/Frq,Particulars,Reg,Date
1,4260,1202,1209,1218,G14,5,XRA,1-8A,B1.9 1.4E-04,,2010-01-01 12:09:00
3,4270,1233,1243,1300,G14,5,XRA,1-8A,B2.3 3.1E-04,,2010-01-01 12:43:00
5,4280,2329,2333,2342,G14,5,XRA,1-8A,B1.1 8.9E-05,,2010-01-01 23:33:00
7,4290 +,0310,0313,0319,////,5,XRA,1-8A,B1.1 4.4E-05,1039,2010-01-02 03:13:00
9,4300,0709,0724,0740,G14,5,XRA,1-8A,C1.0 1.2E-03,1039,2010-01-02 07:24:00
...,...,...,...,...,...,...,...,...,...,...,...
118402,9830 +,2018,2024,2034,G16,5,XRA,1-8A,B4.5 3.9E-04,,2021-12-30 20:24:00
118404,9840 +,2234,2237,2241,G16,5,XRA,1-8A,B4.2 1.5E-04,,2021-12-30 22:37:00
118406,9850 +,2244,2251,2259,G16,5,XRA,1-8A,B4.1 3.4E-04,,2021-12-30 22:51:00
118407,9850,2255,2255,2255,LEA,G,RBR,610,110,,2021-12-30 22:55:00
